# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import operator

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
vowels = {"a","e","i","o","u"}

In [3]:

def vowel_count(clean_word):
    vowels = {"a","e","i","o","u"}
    count = 0
    for i in clean_word:
        if i in vowels:
            count = count+1
    return count/len(clean_word)

In [4]:
# extracting all the words into a list
text_file = open("words_250000_train.txt","r")
full_dictionary = text_file.read().splitlines()
text_file.close()

In [5]:
# on analysisng data we see the maximum number of letters in dataset it 29.
# we can further see that most of the words are not giberish and have meaning
# in the english langugage many words are compopsite of you smaller words that have meaning
# a good algorithm will be to make a collection of all the connected substrings of the words provided

n_word_dictionary = {i:[] for i in range(3, 30)} # max is 29
# minimum length of substring is 3 as there are very less words with only 2 letters that have meaning and are a part of a composite word
count = 3 
while count<=29 :
    for words in full_dictionary:
        if(len(words)>=count):
            for i in range(len(words)-count+1):
                #if words[i:i+count-1] not in n_word_dictionary[count]:
                n_word_dictionary[count].append(words[i:i+count]) 
    count = count+1

In [6]:
# function to generate a list of words which are substring in the original dictionary and of same pattern and length as clean word

def match_substring(n_word_dictionary, clean_word):
    new_dictionary = []
    l = len(clean_word)
    for dict_word in n_word_dictionary[l]:
        if re.match(clean_word,dict_word):
            new_dictionary.append(dict_word)
    return new_dictionary

In [7]:

# bigram model
# bigram_counts_right = {}
# bigram_counts_left = {}

# unigram_counts ={}

# for word in full_dictionary:
#     xword = '@' + word
#     for i in range(len(xword)-1):
#         unigram_unit = xword[i]
#         bigram_unit = (xword[i] , xword[i+1])
#         if bigram_unit in bigram_counts_right:
#             bigram_counts_right[bigram_unit] +=1
#         else:
#             bigram_counts_right[bigram_unit] = 1
        
#         if unigram_unit in unigram_counts:
#             unigram_counts[unigram_unit] += 1
#         else:
#             unigram_counts[unigram_unit] = 1
    
#     yword = word + "*"
#     for i in range(1,len(yword)):
#         bigram_unit = (yword[i-1] , yword[i])
#         if bigram_unit in bigram_counts_left:
#             bigram_counts_left[bigram_unit] +=1
#         else:
#             bigram_counts_right[bigram_unit] =1
            
    
# def calc_prob_bigram(letter_left,letter_right):
#     letter_counts ={}
#     letter_counts_right = {}
#     letter_counts_left ={}
#     total_counts_right = 0
#     total_counts_left = 0
    
#     final =[]
    
#     for bigram in bigram_counts_right:
#         if bigram[0] != letter_left:
#             continue
#         letter_counts_right[bigram[1]] = bigram_counts_right[bigram]
#         total_counts_right += bigram_counts_right[bigram]
        
#     for bigram in bigram_counts_left:
#         if bigram[1] != letter_right:
#             continue
#         letter_counts_left[bigram[0]] = bigram_counts_left[bigram]
#         total_counts_left += bigram_counts_left[bigram]
        
#     sorted_letter_counts_left = dict(sorted(letter_counts_left.items(), key=operator.itemgetter(1),reverse=True))
#     sorted_letter_counts_right = dict(sorted(letter_counts_right.items(), key=operator.itemgetter(1),reverse=True))
    
#     final_prob = {}
    
#     if letter_left == "@" or letter_right==".":
#         for letter_set in sorted_letter_counts_right:
#             prob_letter = (sorted_letter_counts_right[letter_set]/total_counts_right)
            
#             if prob_letter > 0.0 :
#                 final_prob[letter_set] = prob_letter
#         sorted_final_prob = dict(sorted(final_prob.items(), key=operator.itemgetter(1),reverse=True))
        
#         for i in sorted_final_prob:
#             final.append((i,sorted_final_prob[i]))
        
#         return final
        
#     if letter_right == "*" or letter_left == ".":
#         for letter_set in sorted_letter_counts_left:
#             prob_letter = sorted_letter_counts_left[letter_set]/total_counts_left
#             if prob_letter > 0.0 :
#                 final_prob[letter_set] = prob_letter
#         sorted_final_prob = dict(sorted(final_prob.items(), key=operator.itemgetter(1),reverse=True))
        
#         for i in sorted_final_prob:
#             final.append((i,sorted_final_prob[i]))
        
#         return final 
    
#     if letter_right == "." and letter_left == "." :
#         return final

#     for letter_set in sorted_letter_counts_right:
#         prob_letter = (sorted_letter_counts_right[letter_set]/total_counts_right)*0.5
#         if letter_set in sorted_letter_counts_left:
#             prob_letter += (sorted_letter_counts_left[letter_set]/total_counts_left)*0.5
#         if prob_letter > 0.0 :
#             final_prob[letter_set] = prob_letter
    
#     for letter_set in sorted_letter_counts_left:
#         if letter_set not in sorted_letter_counts_right:
#             prob_letter = sorted_letter_counts_left[letter_set]*0.5/total_counts_left
#         if prob_letter > 0.0 :
#             final_prob[letter_set] = prob_letter
            
            
#     sorted_final_prob = dict(sorted(final_prob.items(), key=operator.itemgetter(1),reverse=True))
    
#     for i in sorted_final_prob:
#         final.append((i,sorted_final_prob[i]))
    
#     return final
        
    

In [8]:
# trigam model

# trigram_counts = {}

# for word in full_dictionary:
#     word = "@@" + word + "**"
#     for i in range(len(word)-2):
#         trigram = word[i:i+3]
#         if trigram in trigram_counts:
#             trigram_counts[trigram] += 1
#         else:
#             trigram_counts[trigram] = 1
    
# def trigram_prob(clean_word,i):
    
#     if len(clean_word)<3 :
#         return []
        
#     set1 = {}
#     set2 = {}
#     set3 = {}
    
#     final =[]
    
#     total_1 = 0
#     total_2 = 0
#     total_3 = 0
#     if i == 0 and clean_word[i+1] != ".":
#         for trigram in trigram_counts:
#             if trigram[0] == "@"  and trigram[2] == clean_word[i+1] and trigram[1] != "@" :
#                 set2[trigram[1]] = trigram_counts[trigram]
#                 total_2 += trigram_counts[trigram]
        
#         sorted_set2 = dict( sorted(set2.items(), key=operator.itemgetter(1),reverse=True))
        
#         for unit in sorted_set2 :
#             p = set2[unit]/total_2
#             if p > 0.0 :
#                 final.append((unit,p))
        
#         return final

        
        
#     if i == 1 :
#         if clean_word[i+1] == "." or clean_word[i+2] == "." :
#             return []
#         else:
#             bi = clean_word[i+1] + clean_word[i+2]
#             for trigram in trigram_counts:
#                 if trigram[1:3] == bi and trigram[0] != "@":
#                     set1[trigram[0]] = trigram_counts[trigram]
#                     total_1 += trigram_counts[trigram]
            
#         sorted_set1 = dict( sorted(set1.items(), key=operator.itemgetter(1),reverse=True))

#         for unit in sorted_set1 :
#             p = set1[unit]/total_1
#             if p > 0.0 :
#                 final.append((unit,p))
        
#         return final
        
#     if i >= len(clean_word)-2:
#         if clean_word[i-1] == "." or clean_word[i-2] == "." :
#             return []
#         else:
#             bi = clean_word[i-2] + clean_word[i-1]
#             for trigram in trigram_counts:
#                 if trigram[0:2] == bi and trigram[2] != "*" :
#                     set3[trigram[2]] = trigram_counts[trigram]
#                     total_3 += trigram_counts[trigram]
            
#         sorted_set3 = dict( sorted(set3.items(), key=operator.itemgetter(1),reverse=True))

#         for unit in sorted_set3 :
#             p = set3[unit]/total_3
#             if p > 0.0 :
#                 final.append((unit,p))
        
#         return final
    
#     set1 = {}
#     set2 = {}
#     set3 = {}

#     final = []
    
#     total_1 = 0
#     total_2 = 0
#     total_3 = 0
    
#     if clean_word[i+1] != "." and clean_word[i+2] != ".":
#         bi = clean_word[i+1] + clean_word[i+2]
#         for trigram in trigram_counts:
#             if trigram[1:3] == bi and trigram[0] != "@":
#                 set1[trigram[0]] = trigram_counts[trigram]
#                 total_1 += trigram_counts[trigram]
            
#         set1 = dict( sorted(set1.items(), key=operator.itemgetter(1),reverse=True))
        
#     if clean_word[i-1] != "." and clean_word[i-2] != ".":
#         bi = clean_word[i-2] + clean_word[i-1]
#         for trigram in trigram_counts:
#             if trigram[0:2] == bi and trigram[0] != "*":
#                 set3[trigram[2]] = trigram_counts[trigram]
#                 total_3 += trigram_counts[trigram]
            
#         set3 = dict( sorted(set3.items(), key=operator.itemgetter(1),reverse=True))
        
#     if clean_word[i-1] =="." and clean_word[i+1] == ".":
#         for trigram in trigram_counts:
#             if trigram[0] == clean_word[i-1]  and trigram[2] == clean_word[i+1] :
#                 set2[trigram[1]] = trigram_counts[trigram]
#                 total_2 += trigram_counts[trigram]
            
#         set2 = dict( sorted(set2.items(), key=operator.itemgetter(1),reverse=True))
    
#     if set1 != {} and set3 != {} and set2 != {}:
#         for unit in set1 :
#             p = set1[unit]/total_1
#             if unit in set2 :
#                 p += set2[unit]/total_2
#             if unit in set3:
#                 p += set3[unit]/total_3
#             if p/3 > 0.0 :
#                 final.append((unit,p/3))
    
#     elif set1 == {} and set2 != {} and set3 != {}:
#         for unit in set2 :
#             p = set2[unit]/total_2
#             if unit  in set3:
#                 p += set3[unit]/total_3
            
#             if p/2 > 0.0 :
#                 final.append((unit,p/2))
    
#     elif set3 == {} and set2 != {} and set1 != {}:
#         for unit in set2 :
#             p = set2[unit]/total_2
#             if unit  in set1:
#                 p += set1[unit]/total_1
            
#             if p/2 > 0.0 :
#                 final.append((unit,p/2))
    
#     elif set2 == {} and set1 != {}:
#         for unit in set1 :
#             p = set1[unit]/total_1
#             if p > 0.0 :
#                 final.append((unit,p))
            
#     elif set2 =={} and set3 != {}:
#         for unit in set3 :
#             p = set3[unit]/total_3
#             if p > 0.0 :
#                 final.append((unit,p)) 
    
#     elif set2 != {} and set1 == {} and set3 == {}:
#         for unit in set2 :
#             p = set2[unit]/total_2
#             if p > 0.0 :
#                 final.append((unit,p))
            
#     return final

In [9]:
# tetra-gram

# tetragram_counts = {}

# for word in full_dictionary:
#     word = "@@@" + word + "***"
#     for i in range(len(word)-3):
#         tetragram = word[i:i+3]
#         if tetragram in tetragram_counts:
#             tetragram_counts[trigram] += 1
#         else:
#             tetragram_counts[trigram] = 1
            
# def tetragram_prob(clean_word,i):
     
#     set_1 = {}
#     set_2 = {}
#     set_3 = {}
#     set_4 = {}
    
#     final =[]
    
#     total_1 = 0
#     total_2 = 0
#     total_3 = 0
#     total_4 = 0
    
#     if i == 0 :
#         if clean_word[i+1] != ".":
#             for tetragram in tetragram_counts:
#                 if tetragram[0] == "@" and tetragram[1] == "@" and tetragram[2] != "@" :
#                     set_2[tetragram[2]] = tetragram_counts[tetragram]
#                     total_2 += tetragram_counts[tetragram]
            
#             set_2 = dict( sorted(set_2.items(), key=operator.itemgetter(1),reverse=True))
    
#             if clean_word[i+2] != "." :
#                 for tetragram in tetragram_counts:
#                     if tetragram[0] == "@" and tetragram[1] != "@" :
#                         set_3[tetragram[1]] = tetragram_counts[tetragram]
#                         total_3 += tetragram_counts[tetragram]
                 
#                 set_3 = dict( sorted(set_3.items(), key=operator.itemgetter(1),reverse=True))
                 
#         if set_2 != {} and set_3 != {} :
                
#             for unit in set_2 :
#                 p = set_2[unit]/total_2 *0.5
#                 if unit in set_3:
#                     p += set_3[unit]/total_3 * 0.5
                    
#                 final.append((unit,p))
                      
#             return final
        
#         return final
        
#     if len(clean_word) <4:
#         return []
    
#     word = "@@@" + clean_word + "***"
    
#     i += 3
    
#     set_1 = {}
#     set_2 = {}
#     set_3 = {}
#     set_4 = {}
    
#     final =[]
    
#     total_1 = 0
#     total_2 = 0
#     total_3 = 0
#     total_4 = 0
    
#     if word[i-3] != "." and word[i-2] != "." and word[i-1] != "." :
#         for tetragram in tetragram_counts :
#             trio = tetragram[0:3] 
#             if trio == word[i-3:i] and tetragram[3] != "@" and tetragram[3] != "*":
#                 set_1[tetragram[3]] = tetragram_counts[tetragram]
#                 total_1 += tetragram_counts[tetragram]
                
#         set_1 = dict( sorted(set_1.items(), key=operator.itemgetter(1),reverse=True))

#     if word[i-2] != "." and word[i-1] != "." and word[i+1] != "." :
#         for tetragram in tetragram_counts :
#             bi = tetragram[0:2] 
#             if bi == word[i-2:i] and tetragram[3] == word[i+1] and tetragram[2] != "*" and tetragram[2] != "@" :
#                 set_2[tetragram[2]] = tetragram_counts[tetragram]
#                 total_2 += tetragram_counts[tetragram]
                
#         set_2 = dict( sorted(set_2.items(), key=operator.itemgetter(1),reverse=True))
       
#     if word[i-1] != "." and word[i+1] != "." and word[i+2] != "." :
#         for tetragram in tetragram_counts :
#             bi = tetragram[2:4] 
#             if bi == word[i+1:i+3] and tetragram[0] == word[i-1] and tetragram[1] != "*" and tetragram[1] != "@" :
#                 set_3[tetragram[1]] = tetragram_counts[tetragram]
#                 total_3 += tetragram_counts[tetragram]
                
#         set_3 = dict( sorted(set_3.items(), key=operator.itemgetter(1),reverse=True))
        
#     if word[i+3] != "." and word[i+2] != "." and word[i+1] != "." :
#         for tetragram in tetragram_counts :
#             trio = tetragram[1:4] 
#             if trio == word[i+1:i+4] and tetragram[0] != "@" and tetragram[0] != "*":
#                 set_4[tetragram[0]] = tetragram_counts[tetragram]
#                 total_4 += tetragram_counts[tetragram]
                
#         set_4 = dict( sorted(set_4.items(), key=operator.itemgetter(1),reverse=True))
        
#     list_set = [set_1,set_2,set_3,set_4]
    
#     empty_count = list_set.count({})
#     if empty_count == 4 :
#         return []

#     alphabets = string.ascii_lowercase
    
#     prob_letter = {}
    
#     for alpha in alphabets :
#         p = 0
#         if alpha in set_1 :
#             p += set_1[alpha]/total_1
#         if alpha in set_2 :
#             p += set_2[alpha]/total_2
#         if alpha in set_3 :
#             p += set_3[alpha]/total_3
#         if alpha in set_4 :
#             p += set_4[alpha]/total_4
        
#         prob_letter[alpha] = p/(4-empty_count)
        
#     prob_letter = dict( sorted(prob_letter.items(), key=operator.itemgetter(1),reverse=True))
    
#     for letter in prob_letter :
#         final.append((letter,prob_letter[letter]))
    
#     return final

In [10]:
# pentagram model

# pentagram_counts = {}

# for word in full_dictionary:
#     word = "@@@@" + word + "****"
#     for i in range(len(word)-4):
#         pentagram = word[i:i+3]
#         if pentagram in pentagram_counts:
#             pentagram_counts[pentagram] += 1
#         else:
#             pentagram_counts[pentagram] = 1
            
# def pentagram_prob(clean_word,i):
    
#     set_1 = {}
#     set_2 = {}
#     set_3 = {}
#     set_4 = {}
#     set_5 = {}
    
#     final =[]
    
#     total_1 = 0
#     total_2 = 0
#     total_3 = 0
#     total_4 = 0
#     total_5 = 0
    
#     word = "@@@@" + clean_word + "****"
    
#     i += 4
    
#     if word[i-4] != "." and word[i-3] != "." and word[i-2] != "." and word[i-1] != "." :
#         for pentagram in pentagram_counts :
#             tetro = pentagram[0:4] 
#             if tetro == word[i-4:i] and pentagram[4] != "@" and pentagram[4] != "*":
#                 set_1[pentagram[4]] = pentagram_counts[pentagram]
#                 total_1 += pentagram_counts[pentagram]
                
#         set_1 = dict( sorted(set_1.items(), key=operator.itemgetter(1),reverse=True))
      
#     if word[i+1] != "." and word[i-3] != "." and word[i-2] != "." and word[i-1] != "." :
#         for pentagram in pentagram_counts :
#             trio = pentagram[0:3] 
#             if trio == word[i-4:i] and pentagram[3] != "@" and pentagram[3] != "*" and pentagram[4] == word[i+1] :
#                 set_2[pentagram[3]] = pentagram_counts[pentagram]
#                 total_2 += pentagram_counts[pentagram]
                
#         set_2 = dict( sorted(set_2.items(), key=operator.itemgetter(1),reverse=True))  
    
#     if word[i+1] != "." and word[i+2] != "." and word[i-2] != "." and word[i-1] != "." :
#         for pentagram in pentagram_counts :
#             bi_1 = pentagram[0:2]
#             bi_2 = pentagram[3:5] 
#             if bi_1 == word[i-2:i] and pentagram[2] != "@" and pentagram[2] != "*" and bi_2 == word[i+1:i+3] :
#                 set_3[pentagram[2]] = pentagram_counts[pentagram]
#                 total_3 += pentagram_counts[pentagram]
                
#         set_3 = dict( sorted(set_3.items(), key=operator.itemgetter(1),reverse=True)) 
    
#     if word[i+1] != "." and word[i+2] != "." and word[i+3] != "." and word[i-1] != "." :
#         for pentagram in pentagram_counts :
#             trio = pentagram[2:5] 
#             if trio == word[i+1:i+4] and pentagram[1] != "@" and pentagram[1] != "*" and pentagram[0] == word[i-1] :
#                 set_4[pentagram[3]] = pentagram_counts[pentagram]
#                 total_4 += pentagram_counts[pentagram]
                
#         set_4 = dict( sorted(set_4.items(), key=operator.itemgetter(1),reverse=True))
        
#     if word[i+4] != "." and word[i+3] != "." and word[i+2] != "." and word[i+1] != "." :
#         for pentagram in pentagram_counts :
#             tetro = pentagram[1:5] 
#             if tetro == word[i+1:i+5] and pentagram[0] != "@" and pentagram[0] != "*":
#                 set_5[pentagram[0]] = pentagram_counts[pentagram]
#                 total_5 += pentagram_counts[pentagram]
                
#         set_5 = dict( sorted(set_5.items(), key=operator.itemgetter(1),reverse=True))
    
    
#     list_set = [set_1,set_2,set_3,set_4,set_5]
    
#     empty_count = list_set.count({})
#     if empty_count == 5 :
#         return []

#     alphabets = string.ascii_lowercase
    
#     prob_letter = {}
    
#     for alpha in alphabets :
#         p = 0
#         if alpha in set_1 :
#             p += set_1[alpha]/total_1
#         if alpha in set_2 :
#             p += set_2[alpha]/total_2
#         if alpha in set_3 :
#             p += set_3[alpha]/total_3
#         if alpha in set_4 :
#             p += set_4[alpha]/total_4
#         if alpha in set_5 :
#             p += set_5[alpha]/total_5
#         prob_letter[alpha] = p/(5-empty_count)
        
#     prob_letter = dict( sorted(prob_letter.items(), key=operator.itemgetter(1),reverse=True))
    
#     for letter in prob_letter :
#         final.append((letter,prob_letter[letter]))
    
#     return final
        
    


In [11]:
def get_letter_frequency(new_dictionary):
    
    possible_guess={}
    
    # searches for a match of the exact word in the available word list
    for word in new_dictionary:
        # set_word = set(word)
        for i in word:
            if i in possible_guess:
                possible_guess[i] += 1
            else:
                possible_guess[i] = 1
    
    # sorting dictionary in descending order of frequency of occurence
    sorted_possible_guess = dict(sorted(possible_guess.items(), key=operator.itemgetter(1),reverse=True))
    
    return sorted_possible_guess

In [12]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = {"@","*"}
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = get_letter_frequency(self.full_dictionary)
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        blanks = clean_word.count(".")
        blank_index = []
        for i in range(len(clean_word)):
            if clean_word[i] == ".":
                blank_index.append(i)
        
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        possible_guess={}
        
        # searches for a match of the exact word in the available word list
        for word in self.current_dictionary:
            set_word = set(word)
            for i in set_word:
                if i in possible_guess:
                    possible_guess[i] += 1
                else:
                    possible_guess[i] = 1
                    
        # sorting dictionary in descending order
        sorted_possible_guess = dict(sorted(possible_guess.items(), key=operator.itemgetter(1),reverse=True))
              
        guess_letter = '!'
              
        for letter in sorted_possible_guess:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
                
        
        # # count occurrence of all characters in possible word matches
        # full_dict_string = "".join(new_dictionary)
        
        # c = collections.Counter(full_dict_string)
        # sorted_letter_count = c.most_common()                   
        
        
        
        # most probable match of letter        
            
        # if no word matches in training dictionary, check for words available in the n_word_dictionary

        # if guess_letter == '!':
        #     new_dictionary = match_substring(n_word_dictionary, clean_word)
        #     letter_set = get_letter_frequency(new_dictionary)
        #     for letter in letter_set:
        #         if letter not in self.guessed_letters:
        #             if letter in vowels and vowel_count(clean_word)>0.55:
        #                 self.guessed_letters.append(letter)
        #                 continue
        #             guess_letter = letter
        #             break
                
        
            
        # if no substring matches split word into substrnigs and match patterns with substrings of other words
        
        # split in half
        
        substring_len = len(clean_word) -1
        
        letter_frq ={}
        while guess_letter == "!" :
            for i in range(len(clean_word)-substring_len+1):
                s = clean_word[i:i+substring_len]
                new_dictionary = match_substring(n_word_dictionary, s)
                letter_set = get_letter_frequency(new_dictionary)
                for letter in letter_set:
                    if letter not in self.guessed_letters:
                        if letter in letter_frq:
                            letter_frq[letter] += letter_set[letter]
                        else:
                            letter_frq[letter] = letter_set[letter]
            
            sorted_letter_frq = dict(sorted(letter_frq.items(), key=operator.itemgetter(1),reverse=True))
            
            for letter in sorted_letter_frq :
                guess_letter = letter
                break
            
            substring_len -= 1
        
        # if guess_letter == "!":
            
        #     most_probable_letter_pentagram = {}
        #     pentagram_predictions = []
            
        #     for i in blank_index:
        #         pentagram_predictions = pentagram_prob(clean_word,i)
                
        #         if pentagram_predictions == []:
        #             continue
                
        #         for letter_prob_set in pentagram_predictions:
        #             if letter_prob_set[0] in self.guessed_letters:
        #                 continue
        #             else:
        #                 most_probable_letter_pentagram[letter_prob_set[0]] = letter_prob_set[1] 
                
        #     sorted_most_probable_letter = dict(sorted(most_probable_letter_pentagram.items(), key=operator.itemgetter(1),reverse=True))

        #     for letter in sorted_most_probable_letter:
        #         print("Pentagram :",letter)
        #         guess_letter = letter
        #         break
        
         
        # if guess_letter == "!":
            
        #     most_probable_letter_tretragram = {}
        #     tetragram_predictions = []
            
        #     for i in blank_index:
        #         tetragram_predictions = tetragram_prob(clean_word,i)
                
        #         if tetragram_predictions == []:
        #             continue
                
        #         for letter_prob_set in tetragram_predictions:
        #             if letter_prob_set[0] in self.guessed_letters:
        #                 continue
        #             else:
        #                 most_probable_letter_tretragram[letter_prob_set[0]] = letter_prob_set[1] 
                
        #     sorted_most_probable_letter = dict(sorted(most_probable_letter_tretragram.items(), key=operator.itemgetter(1),reverse=True))

        #     for letter in sorted_most_probable_letter:
        #         print("Tetragram :",letter)
        #         guess_letter = letter
        #         break
                
                   
        # if guess_letter == "!":
            
        #     most_probable_letter_trigram ={}
        #     most_probable_letter = {}
        #     trigram_predictions =[]
        #     bigram_predictions = []
            
        #     for i in blank_index:
                
        #         trigram_predictions = trigram_prob(clean_word,i)
                
        #         if trigram_predictions == [] :
        #             continue    
                
        #         for letter_prob_set in trigram_predictions:
        #             if letter_prob_set[0] in self.guessed_letters:
        #                 continue
        #             else:
        #                 most_probable_letter_trigram[letter_prob_set[0]] = letter_prob_set[1] 
                                
        #     sorted_most_probable_letter = dict(sorted(most_probable_letter_trigram.items(), key=operator.itemgetter(1),reverse=True))
        #     if "*" in sorted_most_probable_letter:
        #         sorted_most_probable_letter.pop("*")
            
        #     if "@" in sorted_most_probable_letter:
        #         sorted_most_probable_letter.pop("*")
                
        #     for letter in sorted_most_probable_letter:
        #         print("Trigram :",letter)
        #         guess_letter = letter
        #         break
            
                            
        # if guess_letter == "!" :
        #     most_probable_letter = ("!",0)
        #     bigram_predictions = []
        #     for i in blank_index:
        #         if i == 0 :
        #             bigram_predictions = calc_prob_bigram("@",clean_word[i+1]) 
        #         elif i < len(clean_word)-1:
        #             bigrm_predictions = calc_prob_bigram(clean_word[i-1],clean_word[i+1])
        #         else :
        #             bigram_predictions = calc_prob_bigram(clean_word[i-1],"*")
                    
        #         for letter_prob_set in bigram_predictions:
        #             if letter_prob_set[0] in self.guessed_letters:
        #                 continue
        #             else:
        #                 if most_probable_letter[1] < letter_prob_set[1]:
        #                     most_probable_letter = letter_prob_set
            
            
        #     guess_letter = most_probable_letter[0]
        #     if guess_letter != "!":
        #         print("Biagram :",guess_letter)
                    
        # # if still no letter matches use the default letter finding system using number of vowels to number of letters ratio    
             
        # if guess_letter == '!':
        #     sorted_letter_count = self.full_dictionary_common_letter_sorted
        #     for letter,  in sorted_letter_count:
        #         if letter not in self.guessed_letters:
        #             if letter in vowels and vowel_count(clean_word)>0.55:
        #                 self.guessed_letters.append(letter)
        #                 continue
        #             guess_letter = letter
        #             break             
        
        self.guessed_letters.append(guess_letter)
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [13]:
api = HangmanAPI(access_token="491e016af428d59d8ca34710520d7e", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [14]:
# for i in range(500):
#     api.start_game(practice=1,verbose=True)
#     [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
#     practice_success_rate = total_practice_successes / total_practice_runs
#     print(total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes)
#     print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [22]:
for i in range(138):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [23]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
print(total_recorded_runs,total_recorded_successes)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

1000 564
overall success rate = 0.564
